In [26]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.utils import resample

# Data

In [27]:
np.random.seed(2)

dataset_size = ["small", "medium", "large", "beta", "alpha"][0]

dataset_info = {
    "small": {
        "dataset_name": "wine",
        "class_name": "Class",
        "drop_fields": []
    },
    "medium": {
        "dataset_name": "breast-cancer-wisconsin",
        "class_name": "Class",
        "drop_fields": ["Sample code number"]
    },
    "large": {
        "dataset_name": "seismic-bumps",
        "class_name": "class",
        "drop_fields": []
    },
    "beta": {
        "dataset_name": "Acoustic_Extinguisher_Fire_Dataset",
        "class_name": "Class",
        "drop_fields": []
    },
    "alpha": {
        "dataset_name": "ThoracicSurgery",
        "class_name": "Risk1Yr",
        "drop_fields": []
    }
}

dataset_name = dataset_info[dataset_size]["dataset_name"]
class_name = dataset_info[dataset_size]["class_name"]
drop_fields = dataset_info[dataset_size]["drop_fields"]

df = pd.read_csv('../data/' + dataset_name + ".csv")
df = df.drop(drop_fields, axis=1)
df = df.iloc[np.random.permutation(len(df))]

if dataset_name == "breast-cancer-wisconsin":
    df[class_name].replace({2: 0, 4: 1}, inplace=True)
    
if dataset_name == "ThoracicSurgery":
    df[class_name].replace({'T': 1, 'F': 0}, inplace=True)

n_cut = int(0.8*len(df))
df_trn = df[:n_cut]
df_tst = df[n_cut:]

if dataset_name == "ThoracicSurgery":
    df_1 = df_trn[df_trn[class_name] == 1]
    df_0 = df_trn[df_trn[class_name] != 1]  
    df_1_upsampled = resample(df_1, random_state=2, n_samples=len(df_0), replace=True)

    df_upsampled = pd.concat([df_1_upsampled, df_0], ignore_index=True)
    df_upsampled = df_upsampled.iloc[np.random.permutation(df_upsampled.index)]
    
    df_0_downsampled = resample(df_0, random_state=20, n_samples=len(df_1), replace=False)
    df_downsampled = pd.concat([df_0_downsampled, df_1], ignore_index=True)
    df_downsampled = df_downsampled.iloc[np.random.permutation(df_downsampled.index)]
    
    # df_trn = df_upsampled
    df_trn = df_downsampled

X_trn = df_trn.drop(class_name, axis=1)
y_trn = df_trn[class_name]

X_tst = df_tst.drop(class_name, axis=1)
y_tst = df_tst[class_name]

# Random Forest

In [28]:
from RandomForest_df import RandomForest_df

M = X_trn.shape[1]
CV_dict_params = {'NT': [1, 10, 25, 50, 75, 100],
                  'F': sorted(list(set([1, 3, int(np.log2(M + 1)), int(np.sqrt(M))])))
                  }

best_F, best_NT, best_score = None, None, -1
all_metrics_CV = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        all_scores = np.zeros(5)
        N = len(X_trn)
        for run in range(5):
            
            ind_tst_ = np.full(len(X_trn), False)
            ind_tst_[int(N*run/5):int(N*(run+1)/5)] = True
            
            X_trn_, y_trn_ = X_trn[ind_tst_], y_trn[ind_tst_]
            X_tst_, y_tst_ = X_trn[~ind_tst_], y_trn[~ind_tst_]
            
            clf = RandomForest_df(NT=NT_, F=F_)
            clf.fit(X_trn_, y_trn_)
            if dataset_name == "wine":
                all_scores[run] = accuracy_score(clf.predict(X_tst_), y_tst_)
            else:
                all_scores[run] = clf.score(X_tst_, y_tst_)
            
        score_ = all_scores.mean()
        all_metrics_CV.append(score_)
        
        if score_ > best_score:
            best_F, best_NT, best_score = F_, NT_, score_

        if dataset_name == "wine":
            print(f'(F, NT) = {(F_, NT_)} \t--> \t Accuracy = {round(score_, 3)}')
        else:
            print(f'(F, NT) = {(F_, NT_)} \t--> \t F1-Score = {round(score_, 3)}')
        
all_metrics_CV = np.array(all_metrics_CV)


(F, NT) = (1, 1) 	--> 	 Accuracy = 0.736
(F, NT) = (1, 10) 	--> 	 Accuracy = 0.882
(F, NT) = (1, 25) 	--> 	 Accuracy = 0.937
(F, NT) = (1, 50) 	--> 	 Accuracy = 0.944
(F, NT) = (1, 75) 	--> 	 Accuracy = 0.947
(F, NT) = (1, 100) 	--> 	 Accuracy = 0.942
(F, NT) = (3, 1) 	--> 	 Accuracy = 0.757
(F, NT) = (3, 10) 	--> 	 Accuracy = 0.903
(F, NT) = (3, 25) 	--> 	 Accuracy = 0.937
(F, NT) = (3, 50) 	--> 	 Accuracy = 0.949
(F, NT) = (3, 75) 	--> 	 Accuracy = 0.945
(F, NT) = (3, 100) 	--> 	 Accuracy = 0.945


In [29]:
print(f'Best Parameters (F, NT): {(best_F, best_NT)}')
best_RF = RandomForest_df(NT=best_NT, F=best_F)
best_RF.fit(X_trn, y_trn, verbose=3)


Best Parameters (F, NT): (3, 50)
Feature importance: Index(['Color intensity', 'Flavanoids', 'OD280/OD315 of diluted wines',
       'Proline', 'Hue', 'Malic acid', 'Alcohol', 'Total phenols', 'Magnesium',
       'Ash', 'Alcalinity of ash', 'Proanthocyanins', 'Nonflavanoid phenols'],
      dtype='object')


RandomForest_df(F=3, NT=50)

In [30]:
y_trn_hat = best_RF.predict(X_trn)
y_tst_hat = best_RF.predict(X_tst)

In [31]:
average = "binary"
if dataset_name in ['wine']:
    average = "macro"
print(average)

print(f'Accuracy (test): {round(accuracy_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'Precision (test): {round(precision_score(y_tst.to_numpy(), y_tst_hat, average="macro"), 3)}')
print(f'Recall (test): {round(recall_score(y_tst.to_numpy(), y_tst_hat, average=average), 3)}')
print(f'F1 Score (test): {round(f1_score(y_tst.to_numpy(), y_tst_hat, average=average), 3)}')
print('-'*15)
print(f'Accuracy (train): {round(accuracy_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'Precision (train): {round(precision_score(y_trn.to_numpy(), y_trn_hat, average=average), 3)}')
print(f'Recall (train): {round(recall_score(y_trn.to_numpy(), y_trn_hat, average=average), 3)}')
print(f'F1 Score (train): {round(f1_score(y_trn.to_numpy(), y_trn_hat, average=average), 3)}')


macro
Accuracy (test): 1.0
Precision (test): 1.0
Recall (test): 1.0
F1 Score (test): 1.0
---------------
Accuracy (train): 1.0
Precision (train): 1.0
Recall (train): 1.0
F1 Score (train): 1.0


In [34]:
all_importances = []
all_metrics = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        clf = RandomForest_df(NT=NT_, F=F_)
        print(f'(F, NT) = {(F_, NT_)}')
        clf.fit(X_trn, y_trn, verbose=3)
        all_importances.append(np.flip(np.argsort(clf.importance)))
        y_tst_hat = clf.predict(X_tst)     
        
        acc = accuracy_score(y_tst.to_numpy(), y_tst_hat)
        prec = precision_score(y_tst.to_numpy(), y_tst_hat, average=average)
        rec = recall_score(y_tst.to_numpy(), y_tst_hat, average=average)
        f1_ = f1_score(y_tst.to_numpy(), y_tst_hat, average=average)
           
        print(f'Accuracy (test): {round(acc, 3)}')
        print(f'Precision (test): {round(prec, 3)}')
        print(f'Recall (test): {round(rec, 3)}')
        print(f'F1 Score (test): {round(f1_, 3)}')
        print('-'*15)
        
        all_metrics.append([acc, prec, rec, f1_])
        
all_importances = np.array(all_importances)
all_metrics = np.array(all_metrics)

(F, NT) = (1, 1)
Feature importance: Index(['Nonflavanoid phenols', 'Hue', 'Alcalinity of ash', 'Proline',
       'Proanthocyanins', 'Flavanoids', 'Total phenols', 'Magnesium',
       'Malic acid', 'OD280/OD315 of diluted wines', 'Color intensity', 'Ash',
       'Alcohol'],
      dtype='object')
Accuracy (test): 0.833
Precision (test): 0.827
Recall (test): 0.863
F1 Score (test): 0.839
---------------
(F, NT) = (1, 10)
Feature importance: Index(['Nonflavanoid phenols', 'OD280/OD315 of diluted wines', 'Malic acid',
       'Proline', 'Proanthocyanins', 'Magnesium', 'Flavanoids', 'Ash',
       'Total phenols', 'Alcohol', 'Hue', 'Alcalinity of ash',
       'Color intensity'],
      dtype='object')
Accuracy (test): 0.944
Precision (test): 0.954
Recall (test): 0.954
F1 Score (test): 0.954
---------------
(F, NT) = (1, 25)
Feature importance: Index(['Magnesium', 'Hue', 'Proline', 'Proanthocyanins',
       'Nonflavanoid phenols', 'Alcohol', 'Color intensity', 'Malic acid',
       'OD280/OD315 o

In [35]:
all_metrics[:, 1][np.argmax(all_metrics_CV)]

1.0

In [36]:
all_metrics_CV

array([0.73563111, 0.88217668, 0.93670238, 0.94373544, 0.94725974,
       0.94199658, 0.75711846, 0.90322931, 0.93668685, 0.9489986 ,
       0.94547431, 0.94547431])

In [37]:
all_metrics_CV

array([0.73563111, 0.88217668, 0.93670238, 0.94373544, 0.94725974,
       0.94199658, 0.75711846, 0.90322931, 0.93668685, 0.9489986 ,
       0.94547431, 0.94547431])

In [38]:
print(all_metrics.T)

[[0.83333333 0.94444444 1.         1.         1.         1.
  0.83333333 1.         0.97222222 1.         0.97222222 0.97222222]
 [0.82702483 0.95396825 1.         1.         1.         1.
  0.84038462 1.         0.97916667 1.         0.97916667 0.97916667]
 [0.86349206 0.95396825 1.         1.         1.         1.
  0.86349206 1.         0.97619048 1.         0.97619048 0.97619048]
 [0.83928571 0.95396825 1.         1.         1.         1.
  0.84887794 1.         0.97690163 1.         0.97690163 0.97690163]]


In [39]:
print(all_metrics_CV.T)

[0.73563111 0.88217668 0.93670238 0.94373544 0.94725974 0.94199658
 0.75711846 0.90322931 0.93668685 0.9489986  0.94547431 0.94547431]


In [40]:
from utils.print_latex import print_table

print("IMPORTANCES")
print_table(all_importances.T)
print("-"*15)

print("ACC - PRECISION - RECALL - F1 (TEST)")
print_table(all_metrics.T)
print("-"*15)

print("F1 (CV)")
print_table(all_metrics_CV)
print("-"*15)


IMPORTANCES
\textbf{\#1} & & 7 & 7 & 4 & 4 & 1 & 7 & 6 & 6 & 6 & 9 & 9 & 9\\
\hline
\textbf{\#2} & & 10 & 11 & 10 & 1 & 4 & 1 & 11 & 9 & 9 & 6 & 6 & 6\\
\hline
\textbf{\#3} & & 3 & 1 & 12 & 0 & 10 & 4 & 9 & 12 & 12 & 11 & 12 & 12\\
\hline
\textbf{\#4} & & 12 & 12 & 8 & 10 & 7 & 6 & 7 & 11 & 11 & 12 & 11 & 11\\
\hline
\textbf{\#5} & & 8 & 8 & 7 & 5 & 6 & 12 & 3 & 4 & 1 & 10 & 1 & 0\\
\hline
\textbf{\#6} & & 6 & 4 & 0 & 6 & 12 & 10 & 1 & 2 & 4 & 1 & 0 & 10\\
\hline
\textbf{\#7} & & 5 & 6 & 9 & 12 & 2 & 2 & 0 & 1 & 2 & 0 & 10 & 1\\
\hline
\textbf{\#8} & & 4 & 2 & 1 & 8 & 5 & 5 & 12 & 0 & 0 & 5 & 5 & 5\\
\hline
\textbf{\#9} & & 1 & 5 & 11 & 2 & 0 & 8 & 10 & 8 & 10 & 4 & 4 & 8\\
\hline
\textbf{\#10} & & 11 & 0 & 2 & 7 & 11 & 3 & 8 & 3 & 3 & 2 & 8 & 4\\
\hline
\textbf{\#11} & & 9 & 10 & 5 & 9 & 8 & 0 & 5 & 10 & 8 & 3 & 2 & 3\\
\hline
\textbf{\#12} & & 2 & 3 & 6 & 11 & 9 & 9 & 4 & 5 & 5 & 8 & 3 & 2\\
\hline
\textbf{\#13} & & 0 & 9 & 3 & 3 & 3 & 11 & 2 & 7 & 7 & 7 & 7 & 7\\
\hline

-----------

In [41]:
from sys import modules
del modules["utils.print_latex"]

# Decision Tree

In [42]:
from DecisionForest_df import DecisionForest_df

M = X_trn.shape[1]
CV_dict_params = {'NT': [1, 10, 25, 50, 75, 100],
                  'F': sorted(list(set([int(M/4), int(M/2), int(3*M/4)]))) + [-1]
                  }

best_F, best_NT, best_score = None, None, -1
all_metrics_CV = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        all_scores = np.zeros(5)
        N = len(X_trn)
        for run in range(5):
            
            ind_tst_ = np.full(len(X_trn), False)
            ind_tst_[int(N*run/5):int(N*(run+1)/5)] = True
            
            X_trn_, y_trn_ = X_trn[ind_tst_], y_trn[ind_tst_]
            X_tst_, y_tst_ = X_trn[~ind_tst_], y_trn[~ind_tst_]
            
            clf = DecisionForest_df(NT=NT_, F=F_)
            clf.fit(X_trn_, y_trn_)
        
            if dataset_name == "wine":
                all_scores[run] = accuracy_score(clf.predict(X_tst_), y_tst_)
            else:
                all_scores[run] = clf.score(X_tst_, y_tst_)
            
        score_ = all_scores.mean()
        all_metrics_CV.append(score_)
        
        if score_ > best_score:
            best_F, best_NT, best_score = F_, NT_, score_

        if dataset_name == "wine":
            print(f'(F, NT) = {(F_, NT_)} \t--> \t Accuracy = {round(score_, 3)}')
        else:
            print(f'(F, NT) = {(F_, NT_)} \t--> \t F1-Score = {round(score_, 3)}')
        
all_metrics_CV = np.array(all_metrics_CV)


(F, NT) = (3, 1) 	--> 	 Accuracy = 0.731
(F, NT) = (3, 10) 	--> 	 Accuracy = 0.831
(F, NT) = (3, 25) 	--> 	 Accuracy = 0.907
(F, NT) = (3, 50) 	--> 	 Accuracy = 0.935
(F, NT) = (3, 75) 	--> 	 Accuracy = 0.931
(F, NT) = (3, 100) 	--> 	 Accuracy = 0.937
(F, NT) = (6, 1) 	--> 	 Accuracy = 0.75
(F, NT) = (6, 10) 	--> 	 Accuracy = 0.923
(F, NT) = (6, 25) 	--> 	 Accuracy = 0.926
(F, NT) = (6, 50) 	--> 	 Accuracy = 0.919
(F, NT) = (6, 75) 	--> 	 Accuracy = 0.917
(F, NT) = (6, 100) 	--> 	 Accuracy = 0.916
(F, NT) = (9, 1) 	--> 	 Accuracy = 0.828
(F, NT) = (9, 10) 	--> 	 Accuracy = 0.893
(F, NT) = (9, 25) 	--> 	 Accuracy = 0.905
(F, NT) = (9, 50) 	--> 	 Accuracy = 0.893
(F, NT) = (9, 75) 	--> 	 Accuracy = 0.889
(F, NT) = (9, 100) 	--> 	 Accuracy = 0.891
(F, NT) = (-1, 1) 	--> 	 Accuracy = 0.854
(F, NT) = (-1, 10) 	--> 	 Accuracy = 0.912
(F, NT) = (-1, 25) 	--> 	 Accuracy = 0.903
(F, NT) = (-1, 50) 	--> 	 Accuracy = 0.914
(F, NT) = (-1, 75) 	--> 	 Accuracy = 0.916
(F, NT) = (-1, 100) 	--> 	 Accu

In [43]:
print(f'Best Parameters (F, NT): {(best_F, best_NT)}')
best_DF = DecisionForest_df(NT=best_NT, F=best_F)
best_DF.fit(X_trn, y_trn, verbose=3)
print(np.flip(np.argsort(best_DF.importance)))

Best Parameters (F, NT): (3, 100)
Feature importance: Index(['Hue', 'Ash', 'Nonflavanoid phenols', 'Alcalinity of ash',
       'Proanthocyanins', 'Flavanoids', 'Malic acid', 'Color intensity',
       'Total phenols', 'Magnesium', 'OD280/OD315 of diluted wines', 'Proline',
       'Alcohol'],
      dtype='object')
[10  2  7  3  8  6  1  9  5  4 11 12  0]


In [44]:
y_trn_hat = best_DF.predict(X_trn)
y_tst_hat = best_DF.predict(X_tst)

In [45]:
average = "binary"
if dataset_name in ['wine']:
    average = "macro"
print(average)

print(f'Accuracy (test): {round(accuracy_score(y_tst.to_numpy(), y_tst_hat), 3)}')
print(f'Precision (test): {round(precision_score(y_tst.to_numpy(), y_tst_hat, average=average), 3)}')
print(f'Recall (test): {round(recall_score(y_tst.to_numpy(), y_tst_hat, average=average), 3)}')
print(f'F1 Score (test): {round(f1_score(y_tst.to_numpy(), y_tst_hat, average=average), 3)}')
print('-'*15)
print(f'Accuracy (train): {round(accuracy_score(y_trn.to_numpy(), y_trn_hat), 3)}')
print(f'Precision (train): {round(precision_score(y_trn.to_numpy(), y_trn_hat, average=average), 3)}')
print(f'Recall (train): {round(recall_score(y_trn.to_numpy(), y_trn_hat, average=average), 3)}')
print(f'F1 Score (train): {round(f1_score(y_trn.to_numpy(), y_trn_hat, average=average), 3)}')


macro
Accuracy (test): 0.944
Precision (test): 0.961
Recall (test): 0.952
F1 Score (test): 0.954
---------------
Accuracy (train): 1.0
Precision (train): 1.0
Recall (train): 1.0
F1 Score (train): 1.0


In [46]:
all_importances = []
all_metrics = []

for F_ in CV_dict_params['F']:
    for NT_ in CV_dict_params['NT']:
        clf = DecisionForest_df(NT=NT_, F=F_)
        print(f'(F, NT) = {(F_, NT_)}')
        clf.fit(X_trn, y_trn, verbose=3)
        y_tst_hat = clf.predict(X_tst)   
        
        all_importances.append(np.flip(np.argsort(clf.importance)))     
        
        acc = accuracy_score(y_tst.to_numpy(), y_tst_hat)
        prec = precision_score(y_tst.to_numpy(), y_tst_hat, average=average)
        rec = recall_score(y_tst.to_numpy(), y_tst_hat, average=average)
        f1_ = f1_score(y_tst.to_numpy(), y_tst_hat, average=average)
           
        print(f'Accuracy (test): {round(acc, 3)}')
        print(f'Precision (test): {round(prec, 3)}')
        print(f'Recall (test): {round(rec, 3)}')
        print(f'F1 Score (test): {round(f1_, 3)}')
        print('-'*15)
        
        all_metrics.append([acc, prec, rec, f1_])
        
all_importances = np.array(all_importances)
all_metrics = np.array(all_metrics)

(F, NT) = (3, 1)
Feature importance: Index(['OD280/OD315 of diluted wines', 'Flavanoids', 'Magnesium', 'Proline',
       'Hue', 'Color intensity', 'Proanthocyanins', 'Nonflavanoid phenols',
       'Total phenols', 'Alcalinity of ash', 'Ash', 'Malic acid', 'Alcohol'],
      dtype='object')
Accuracy (test): 0.944
Precision (test): 0.956
Recall (test): 0.93
F1 Score (test): 0.941
---------------
(F, NT) = (3, 10)
Feature importance: Index(['Ash', 'Hue', 'Magnesium', 'OD280/OD315 of diluted wines', 'Flavanoids',
       'Nonflavanoid phenols', 'Alcalinity of ash', 'Malic acid',
       'Proanthocyanins', 'Total phenols', 'Color intensity', 'Alcohol',
       'Proline'],
      dtype='object')
Accuracy (test): 0.944
Precision (test): 0.954
Recall (test): 0.954
F1 Score (test): 0.954
---------------
(F, NT) = (3, 25)
Feature importance: Index(['Ash', 'Nonflavanoid phenols', 'Alcalinity of ash', 'Malic acid', 'Hue',
       'OD280/OD315 of diluted wines', 'Proline', 'Total phenols',
       'Proant

In [47]:
print(all_metrics.T)


[[0.94444444 0.94444444 0.94444444 0.94444444 0.94444444 0.94444444
  0.94444444 0.97222222 0.97222222 0.97222222 0.97222222 0.97222222
  0.94444444 0.94444444 0.94444444 0.91666667 0.91666667 0.91666667
  0.91666667 0.94444444 0.91666667 0.94444444 0.94444444 0.94444444]
 [0.95555556 0.95396825 0.96078431 0.96078431 0.96078431 0.96078431
  0.95396825 0.97916667 0.97916667 0.97916667 0.97916667 0.97916667
  0.95396825 0.96078431 0.96078431 0.94444444 0.94444444 0.94444444
  0.94444444 0.96078431 0.94444444 0.96078431 0.96078431 0.96078431]
 [0.93015873 0.95396825 0.95238095 0.95238095 0.95238095 0.95238095
  0.95396825 0.97619048 0.97619048 0.97619048 0.97619048 0.97619048
  0.95396825 0.95238095 0.95238095 0.92857143 0.92857143 0.92857143
  0.92857143 0.95238095 0.92857143 0.95238095 0.95238095 0.95238095]
 [0.9406425  0.95396825 0.95352564 0.95352564 0.95352564 0.95352564
  0.95396825 0.97690163 0.97690163 0.97690163 0.97690163 0.97690163
  0.95396825 0.95352564 0.95352564 0.92969697

In [48]:
print(all_metrics_CV.T)

[0.73064741 0.83115976 0.90675361 0.93493246 0.93143922 0.93673343
 0.75006986 0.92255861 0.92609843 0.91903431 0.91729545 0.91552554
 0.82760441 0.89259432 0.90499922 0.89260984 0.88910107 0.89085546
 0.85396678 0.91203229 0.90316721 0.91378668 0.91552554 0.91201677]


In [49]:
from utils.print_latex import print_table

print("IMPORTANCES")
print_table(all_importances.T)
print("-"*15)

print("ACC - PRECISION - RECALL - F1 (TEST)")
print_table(all_metrics.T)
print("-"*15)

print("F1 (CV)")
print_table(all_metrics_CV)
print("-"*15)


IMPORTANCES
\textbf{\#1} & & 11 & 2 & 2 & 2 & 2 & 10 & 6 & 10 & 2 & 10 & 10 & 10 & 2 & 11 & 6 & 6 & 6 & 6 & 12 & 2 & 2 & 2 & 2 & 2\\
\hline
\textbf{\#2} & & 6 & 10 & 7 & 10 & 10 & 2 & 10 & 2 & 10 & 2 & 6 & 6 & 11 & 6 & 10 & 12 & 12 & 11 & 11 & 11 & 11 & 11 & 11 & 11\\
\hline
\textbf{\#3} & & 4 & 4 & 3 & 7 & 3 & 7 & 11 & 6 & 6 & 9 & 2 & 9 & 9 & 10 & 12 & 2 & 11 & 12 & 9 & 7 & 10 & 1 & 10 & 12\\
\hline
\textbf{\#4} & & 12 & 11 & 1 & 1 & 6 & 3 & 8 & 4 & 9 & 6 & 9 & 2 & 6 & 2 & 11 & 11 & 10 & 2 & 6 & 6 & 1 & 5 & 5 & 10\\
\hline
\textbf{\#5} & & 10 & 6 & 10 & 9 & 1 & 8 & 4 & 9 & 12 & 12 & 12 & 12 & 4 & 12 & 2 & 10 & 2 & 9 & 3 & 10 & 6 & 7 & 12 & 6\\
\hline
\textbf{\#6} & & 9 & 7 & 11 & 3 & 9 & 6 & 2 & 1 & 3 & 1 & 1 & 1 & 1 & 9 & 9 & 9 & 9 & 10 & 2 & 5 & 5 & 10 & 1 & 5\\
\hline
\textbf{\#7} & & 8 & 3 & 12 & 11 & 7 & 1 & 12 & 12 & 4 & 11 & 11 & 11 & 12 & 4 & 0 & 4 & 4 & 4 & 10 & 12 & 9 & 6 & 6 & 1\\
\hline
\textbf{\#8} & & 7 & 1 & 5 & 8 & 11 & 9 & 9 & 5 & 1 & 3 & 3 & 3 & 10 & 1 & 4 & 0 & 0 & 